# Описание проекта

Проект об исследовании надежности заемщиков на основе предоставленных банком данных со статистикой платежеспособности клиентов.

**Цель исследования**

Разобраться как разные данные клиентов влияют на их способность погасить кредит в срок.

**Ход исследования**

- Изучение и предобработка данных
- Удаление пропусков
- Обработка аномальных значений
- Изменение типа данных
- Обработка дубликатов
- Проведение категоризации данных
- Исследование подготовленных данных
- Вывод и рекомендации

## Загрузка и изучение данных

In [1]:
# импортируем необходимые библиотеки
import pandas as pd

In [2]:
# загрузим данные
data = pd.read_csv('../data/data.csv')

In [3]:
# выведем первые строки
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


## Предобработка данных

В данном разделе проведем проверку данных на наличие пропусков, дубликатов, аномальных значений, скорректируем типы данных.

In [4]:
# выведем общую информацию
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Выведем количество пропущенных значений для каждого столбца.

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения: `days_employed` и `total_income`. Удалим пропуски в `total_income`.

In [6]:
# заполним пропуски доходов медианным значением с учетом типа клиента
for t in data['income_type'].unique():
    mask = (data['income_type'] == t) & (data['total_income'].isna())
    income_median = data.loc[data['income_type'] == t, 'total_income'].median()
    data.loc[mask, 'total_income'] = income_median

При выводе первых строк видно, что данные в столбце `days_employed` имеют в том числе и отрицательные значения. Вероятно они появились в результате ошибки при получении или преобразовании данных. Заменим их на положительные значения.

In [7]:
data['days_employed'] = data['days_employed'].abs()

Выведем медианные значения трудового стажа `days_employed` для каждого типа занятости.

In [8]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получаются аномально большие значения. Вероятно это технические ошибки при получении или обработке данных. Будем это иметь в виду в дальнейшем анализе.

Выведем уникальные значения столбца `children`.

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

Здесь есть два аномальных значения. Удалим содержащие их строки.

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Посмотрим на уникальные значения столбца `children` еще раз.

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

Аномальные данные удалены.

Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.

In [12]:
# заполним пропуски трудового стажа медианным значением с учетом типа клиента
for t in data['income_type'].unique():
    mask = (data['income_type'] == t) & (data['days_employed'].isna())
    days_median = data.loc[data['income_type'] == t, 'days_employed'].median()
    data.loc[mask, 'days_employed'] = days_median

Убедимся, что все пропуски заполнены.

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный.

In [14]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [15]:
# выведем первые строки
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


В столбце `education` видно, что одни и те же значения записаны по-разному. Посмотрим на все уникальные значения.

In [16]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Приведем их к одному виду.

In [17]:
data['education'] = data['education'].str.lower()

In [18]:
# выведм уникальные значения еще раз
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Посмотрим на количество дубликатов.

In [19]:
data.duplicated().sum()

71

Найдены 71 строки-дубликаты. Удалим их.

In [20]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, создадим в датафрейме `data` столбец `total_income_category` с категориями. Для этого создадим функцию.

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [21]:
# функция присвоения категории по доходам
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [22]:
# применим фукнцию к столбцу total_income
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [23]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C


Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [24]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, с категориями:

- `операции с автомобилем`,
- `операции с недвижимостью`,
- `проведение свадьбы`,
- `получение образования`.

In [25]:
# функция присвоения категории по целе кредита
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [26]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

In [27]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


## Исследование данных и ответы на вопросы заказчика

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для того чтобы узнать, есть ли зависимость между количеством детей и возвратом кредита, отфильтруем датафрейм по значениям `debt`. После фильтровки вычислим отношение клиентов имевших задолженность к клиентам не имевших задолженность.

In [28]:
filtered_debt0 = data[data['debt'] == 0]
filtered_debt0 = filtered_debt0.groupby('children')['debt'].count()
filtered_debt0

children
0    13028
1     4364
2     1858
3      303
4       37
5        9
Name: debt, dtype: int64

Получили датафрейм с клиентами не имевшими задолженность, сгруппированный по количеству детей.

In [29]:
filtered_debt1 = data[data['debt'] == 1]
filtered_debt1 = filtered_debt1.groupby('children')['debt'].count()
filtered_debt1

children
0    1063
1     444
2     194
3      27
4       4
Name: debt, dtype: int64

Получили датафрейм с клиентами имевшими задолженность, сгруппированный по количеству детей.

In [30]:
result_children = filtered_debt1 / filtered_debt0
result_children

children
0    0.081593
1    0.101742
2    0.104413
3    0.089109
4    0.108108
5         NaN
Name: debt, dtype: float64

Напишем функцию, которая создает сводную таблицу, по заданному столбцу, считает по нему количество, сумму и среднее и добавляет итоговые данные. Затем производится переименование столбцов и возвращается полученный результат.

In [31]:
def result_ratio(column_analysis):
    result_table = data.pivot_table(index=column_analysis,
                                    values='debt',
                                    aggfunc=['count', 'sum', 'mean'],
                                    margins=True)
    result_table = result_table.rename(columns={'debt': '',
                                                'count': 'total',
                                                'sum': 'debt',
                                                'mean': 'ratio'})
    return result_table

Вызовем созданную функцию с параметром - столбец `children`.

In [32]:
result_ratio('children')

,total,debt,ratio
,,,
children,,,
0,14091,1063,0.075438
1,4808,444,0.092346
2,2052,194,0.094542
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000
All,21331,1732,0.081196


**Вывод:**

По полученным данным, можно сказать что зависимость есть. Чем больше детей, тем больше процент имевших задолженность. Такой вывод можно однозначно сделать для семей, имеющих до 2 детей включительно. Семьи с количеством детей от 3, составляют малое количество от всего массива данных, что не позволяет сделать по ним однозначных выводов.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для того чтобы узнать, есть ли зависимость между семейным положением и возвратом кредита, отфильтруем датафрейм по значениям `debt`. После фильтровки вычислим отношение клиентов имевшим задолженность к клиентам не имевшим задолженность.

In [33]:
filtered_family0 = data[data['debt'] == 0]
filtered_family0 = filtered_family0.groupby('family_status_id')['debt'].count()
filtered_family0

family_status_id
0    11334
1     3749
2      888
3     1105
4     2523
Name: debt, dtype: int64

Получили клиентов сгрупированных по семейному положению, не имевших задолженность.

In [34]:
filtered_family1 = data[data['debt'] == 1]
filtered_family1 = filtered_family1.groupby('family_status_id')['debt'].count()
filtered_family1

family_status_id
0    927
1    385
2     63
3     84
4    273
Name: debt, dtype: int64

Получили клиентов сгрупированных по семейному положению, имевших задолженность.

Выведем список уникальных статусов семейного положения для сопоставления его с `family_status_id`.

In [35]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Статусы семейного положения соответствуют следующим идентификаторам:
- 0 - женат / замужем;
- 1 - гражданский брак;
- 2 - вдовец / вдова;
- 3 - в разводе;
- 4 - не женат / не замужем.

In [36]:
result_family = filtered_family1 / filtered_family0
result_family.sort_values()

family_status_id
2    0.070946
3    0.076018
0    0.081789
1    0.102694
4    0.108205
Name: debt, dtype: float64

Получили список статусов отсортированный по возрастанию процента имевших задолженность.

Вызовем созданную ранее функцию:

In [37]:
result_ratio('family_status_id')

,total,debt,ratio
,,,
family_status_id,,,
0,12261,927,0.075606
1,4134,385,0.093130
2,951,63,0.066246
3,1189,84,0.070648
4,2796,273,0.097639
All,21331,1732,0.081196


**Вывод:**

Исходя из проведенного анализа, можно сказать что статус семейного положения влияет на возврат кредита в срок. Меньше процентов не возврата у клиентов являющихся вдовцами или находящихся в разводе. Причиной этого может быть меньшая финансовая нагрузка, меньшее потребление и экономия. Далее следуют официально закрепленные семейные отношения. После них с заметным отрывом идут семьи живущие в гражданском браке и клиенты не имеющие семейных отношений. Данные две категории вероятно имеют большие показатели по задолженностям по причине отсутствия обязательств, отсутствия планирования бюджета доходов/расходов, менее ответственного подхода к распоряжению своими финансами.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Чтобы определить зависимость возврата кредита от уровня дохода, разобъем датафрейм на три условные части по уровням дохода:
- `small` - уровень дохода до 50000;
- `medium` - уровень дохода от 50000 до 150000;
- `big` - уровень дохода более 150000.

In [38]:
filtered_income_small = data[data['total_income'] <= 50000]
filtered_income_medium = data[(data['total_income'] > 50000) & (data['total_income'] <= 150000)]
filtered_income_big = data[data['total_income'] > 150000]

Создадим датафреймы со сгруппированными ранее по уровням дохода клиентами не имевшими задолженность:

In [39]:
filtered_income_small0 = filtered_income_small[filtered_income_small['debt'] == 0]['debt'].count()
filtered_income_medium0 = filtered_income_medium[filtered_income_medium['debt'] == 0]['debt'].count()
filtered_income_big0 = filtered_income_big[filtered_income_big['debt'] == 0]['debt'].count()

Создадим датафреймы со сгруппированными ранее по уровням дохода клиентами имевшими задолженность:

In [40]:
filtered_income_small1 = filtered_income_small[filtered_income_small['debt'] == 1]['debt'].count()
filtered_income_medium1 = filtered_income_medium[filtered_income_medium['debt'] == 1]['debt'].count()
filtered_income_big1 = filtered_income_big[filtered_income_big['debt'] == 1]['debt'].count()

Выведем данные по группам клиентов и отношением имевших задолженность к не имевшим:

In [41]:
print(filtered_income_small0, filtered_income_small1, filtered_income_small1 / filtered_income_small0)
print(filtered_income_medium0, filtered_income_medium1, filtered_income_medium1 / filtered_income_medium0)
print(filtered_income_big0, filtered_income_big1, filtered_income_big1 / filtered_income_big0)

348 23 0.06609195402298851
10233 950 0.09283690022476303
9018 759 0.08416500332667998


Так же вызовем созданную ранее функцию, передав ей в параметр столбец `total_income_category`:

In [42]:
result_ratio('total_income_category')

,total,debt,ratio
,,,
total_income_category,,,
A,25,2,0.080000
B,5014,354,0.070602
C,15921,1353,0.084982
D,349,21,0.060172
E,22,2,0.090909
All,21331,1732,0.081196


**Вывод:**

Из полученной сводной таблицы видно, что данных для анализа всех категорий не достаточно. Основное количество клиентов принадлежит к группам `B` и `C`. По ним можно сделать вывод что чем меньше доходов у клиента тем больше рисков появления задолженности по кредиту.

### Как разные цели кредита влияют на его возврат в срок?

Чтобы проанализировать как разные цели кредита влияют на его возврат в срок, отфильтруем клиентов по не имеющим задолженность:

In [43]:
filtered_purpose0 = data[data['debt'] == 0]
filtered_purpose0 = filtered_purpose0.groupby('purpose_category')['debt'].count()
filtered_purpose0

purpose_category
операции с автомобилем      3879
операции с недвижимостью    9971
получение образования       3619
проведение свадьбы          2130
Name: debt, dtype: int64

и по клиентам, имевшим задолженность:

In [44]:
filtered_purpose1 = data[data['debt'] == 1]
filtered_purpose1 = filtered_purpose1.groupby('purpose_category')['debt'].count()
filtered_purpose1

purpose_category
операции с автомобилем      400
операции с недвижимостью    780
получение образования       369
проведение свадьбы          183
Name: debt, dtype: int64

Далее, вычислим отношение имевших задолженность к не имевшим задолженность клиентам:

In [45]:
result_purpose = filtered_purpose1 / filtered_purpose0
result_purpose.sort_values()

purpose_category
операции с недвижимостью    0.078227
проведение свадьбы          0.085915
получение образования       0.101962
операции с автомобилем      0.103119
Name: debt, dtype: float64

Вызовем функцию `result_ratio` для столбца `purpose_category`.

In [46]:
result_ratio('purpose_category')

,total,debt,ratio
,,,
purpose_category,,,
операции с автомобилем,4279,400,0.093480
операции с недвижимостью,10751,780,0.072551
получение образования,3988,369,0.092528
проведение свадьбы,2313,183,0.079118
All,21331,1732,0.081196


**Вывод:**

В результате полученных значений, можно сказать следующее.
Клиенты, бравшие кредит для операций с недвижимостью более ответственные чем другие категории. Это указывает на то, что данная группа клиентов подходит к вопросу кредитования обдуманно. Операции с недвижимостью сами по себе требуют большего внимания и контроля, т.к. задействуются крупные суммы денег, что накладывает необходимость тщательно все расчитать перед принятием решения.

Что касается проведения свадьбы, здесь низкий процент задолженностей может быть обусловлен тем, что кредит берется и гасится двумя семьями. Этот вопрос тоже тщательно прорабатывается, а решение по по нему принимается не спонтанно, а взвешенно.

С получением образования ситуация несколько отличается. Категория может включать в себя молодых клиентов, не имеющих своего источника дохода или в связи с незначительнмы опытом работы имеющих небольшой доход. Тогда это может накладывать дополнительные обязательства на родителей, по факту ничего не получающих от такого кредита.

Операции с автомобилем осуществляются, наиболее вероятно, после принятия решения о взятии кредита одним лицом. Клиент во многих случаях принимает единоличное решение о покупке автомобиля. Это снижает полноту оценки возможности взятия кредита. Кроме того, автомобиль все таки можно отнести к предметам роскоши, а решение о его приобретении принимается не настолько обдуманно, как решение о покупке той же квартиры.

## Общий вывод.

Мы получили входные данные от банка со статистикой платежеспособности клиентов. Нужно было разобраться как влияют разные данные клиентов на их способность погасить кредит в срок.

В первую очередь было проведено изучение и предобработка данных, включающая удаление пропусков, обработку аномальных значений, изменение типов данных, обработка дубликатов и категоризация данных.

Исследование подготовленных данных позволило выявить следующие зависимости: риск просрочки напрямую зависит от текущего финансового дохода клиента, имеющейся финансовой нагрузки, официально зарегистрированных отношений и целей получения кредита.

Портрет надежного заемщика представляет из себя семьянина, без детей, с высоким уровнем заработной платы, имеющий цель кредита - покупка недвижимости.

Портрет рискованного заемщика: не состоящий в отношениях, имеющий детей, с невысоким уровнем заработной платы и целью кредита - покупка автомобиля.

Исходя из полученных процентов невозвратов, заказчику рекомендуется в первую очередь обращать внимание на клиентов не состоящих в отношениях, во вторую очередь на количество имеющихся детей.